In [11]:
captcha_processing_output_folder = "extracted_letter_images"

In [12]:
import cv2
import imutils

In [13]:
def resize_image_to_dimensions(image, desired_width, desired_height):
    (h, w) = image.shape[:2]
    if w > h:
        image = imutils.resize(image, width=desired_width)
    else:
        image = imutils.resize(image, height=desired_height)
    padWidth = int((desired_width - image.shape[1]) / 2.0)
    padHeight = int((desired_height - image.shape[0]) / 2.0)
    imageWBorder = cv2.copyMakeBorder(image, padHeight, padHeight, padWidth, padWidth,
        cv2.BORDER_REPLICATE)
    imageWBorderResized = cv2.resize(imageWBorder, (desired_width, desired_height))
    return imageWBorderResized

In [31]:
def readImage(image_file_path):
    img = cv2.imread(image_file_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = resize_image_to_dimensions(img, 20, 20)
    img = np.expand_dims(img, axis=2)
    return(img)

In [32]:
import numpy as np
import os
from imutils import paths
images = []
labels = []

for image_file_path in imutils.paths.list_images(captcha_processing_output_folder):
    image_file = readImage(image_file_path)
    label = image_file_path.split(os.path.sep)[-2]
    images.append(image_file)
    labels.append(label)

In [33]:
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

In [34]:
from sklearn.model_selection import train_test_split
(X_train, X_test, y_train, y_test) = train_test_split(images, labels, test_size=0.3, random_state=11)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 20, 20, 20)        520       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 20)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 10, 10, 50)        25050     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 5, 5, 50)          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1250)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 500)               625500    
_________________________________________________________________
dense_4 (Dense)              (None, 32)                16032     
Total para

In [36]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer().fit(y_train)
y_train = label_binarizer.transform(y_train)
y_test = label_binarizer.transform(y_test)

Train on 29058 samples, validate on 9686 samples
Epoch 1/1
29058/29058 [==============================] - 58s 2ms/step - loss: 0.0669 - acc: 0.9850 - val_loss: 0.0192 - val_acc: 0.9946


In [38]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense
num_classes = 32
NNmodel = Sequential()
NNmodel.add(Conv2D(20, (5, 5), padding="same", input_shape=(20, 20, 1), activation="relu"))
NNmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
NNmodel.add(Conv2D(50, (5, 5), padding="same", activation="relu"))
NNmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
NNmodel.add(Flatten())
NNmodel.add(Dense(512, activation="relu"))
NNmodel.add(Dense(num_classes, activation="softmax"))
NNmodel.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
NNmodel.summary()

In [3]:
NNmodel.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=16, epochs=5, verbose=1)

In [4]:
CAPTCHA = "captcha_images\\NZH2.png"

In [ ]:
def findBoundingRectanglesOfContours(contours):
    letter_bounding_rectangles = []
    for contour in contours:
        (x, y, w, h) = cv2.boundingRect(contour)
        if w / h > 1.25:
            half_width = int(w / 2)
            letter_bounding_rectangles.append((x, y, half_width, h))
            letter_bounding_rectangles.append((x + half_width, y, half_width, h))
        else:
            letter_bounding_rectangles.append((x, y, w, h))
    return letter_bounding_rectangles

def preprocessCAPTCHA(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    grayWBorder = cv2.copyMakeBorder(gray, 8, 8, 8, 8, cv2.BORDER_REPLICATE)
    preprocessed = cv2.threshold(grayWBorder, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
    return grayWBorder, preprocessed

def getCAPTCHAlabel(pathToFile):
    filename = os.path.basename(pathToFile)
    label = filename.split(".")[0]
    return label

def CAPTCHAtoGrayscaleAndBoundingRectangles(captcha_image_file):
    image = cv2.imread(captcha_image_file)
    gray, preprocessed = preprocessCAPTCHA(image)
    contours = cv2.findContours(preprocessed.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0]
    letter_bounding_rectangles = findBoundingRectanglesOfContours(contours)
    letter_bounding_rectangles = sorted(letter_bounding_rectangles, key=lambda x: x[0])
    return gray, letter_bounding_rectangles

In [ ]:
captcha_label = getCAPTCHAlabel(CAPTCHA)
gray, letter_bounding_rectangles = CAPTCHAtoGrayscaleAndBoundingRectangles(CAPTCHA)
predictions = []

In [ ]:
for letter_bounding_rectangle in letter_bounding_rectangles:
    x, y, w, h = letter_bounding_rectangle
    letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]
    letter_image = resize_image_to_dimensions(letter_image, 20, 20)
    letter_image = np.expand_dims(letter_image, axis=2)
    letter_image = np.expand_dims(letter_image, axis=0)
    prediction = NNmodel.predict(letter_image)
    letter = label_binarizer.inverse_transform(prediction)[0]
    predictions.append(letter)

In [ ]:
predicted_captcha_text = "".join(predictions)
print("Predicted CAPTCHA text is: {}".format(predicted_captcha_text))
print("CAPTCHA text is: {}".format(CAPTCHA.split("\\")[-1].split(".")[0]))